Resumen: 

Piero: El calculo de la matriz Rouge-L test + .zip cargarlo en servidor.

Mario: ejecutar experimentos en el servidor, Incorporar validación y eliminar textos duplicados, verificar que el código funcione.




In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
import random

random.seed(0)

## Data

In [4]:
import json as jsonmod
import numpy as np
import random
import time
import pickle
from tqdm import tqdm
import pandas as pd

dataset -> [ ... (imgid, [5 textos] )...]

In [47]:
train_errors= [6188, 6348, 9369, 13475, 17862, 19678, 19938, 20339, 21050, 21204, 21411, 21731, 23560, 27405, 28359, 29971, 30386, 40425, 40560, 42465, 44088, 46072, 48325, 50466, 50719, 53736, 56448, 60110, 61449, 62136, 63143]

test_errors=[4354, 5235, 6471]

val_errors= [1015, 2451, 2899, 5055, 5819]

print(len(train_errors), len(test_errors))

31 3


In [48]:
dataset= []
df= pd.read_csv("ROCO/radiologytraindata.csv")
index=0
for i in range(df.shape[0]):
    if i not in train_errors:
        dict_={"split": "train","imgid": index, "sentids": [index] }
        index+=1
        dataset.append(dict_)
df_v= pd.read_csv("ROCO/radiologyvaldata.csv")
c=1
for i in range(df_v.shape[0]):
    if i not in val_errors:
        dict_={"split": "val","imgid": c + index, "sentids": [c +index] }
        c+=1
        dataset.append(dict_)
df_T= pd.read_csv("ROCO/radiologytestdata.csv")
c_=1
for i in range(df_T.shape[0]):
    if i not in test_errors:
        dict_={"split": "test","imgid": c_ + c + index, "sentids": [c_ +c+ index] }
        c_+=1
        dataset.append(dict_)

In [51]:
all_val_indices_img,all_test_indices_img, all_val_indices_text,all_test_indices_text=[],[],[],[]
all_train_indices_img,all_train_indices_text=[],[]
for i in range(len(dataset)):
  if dataset[i]["split"]=="train":
    all_train_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_train_indices_text.append(t)
    
  if dataset[i]["split"]=="test":
    all_test_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_test_indices_text.append(t)
    
  if dataset[i]["split"]=="val":
    all_val_indices_img.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      all_val_indices_text.append(t)

print("Train-Val-Test")  
print(len(all_train_indices_img), len(all_val_indices_img), len(all_test_indices_img))




dataset_= []
for i in range(len(dataset)):
#solo test
    if dataset[i]["split"]=="test":
  #if dataset[i]["split"]!="test":
    dataset_.append(dataset[i])
dataset= list(dataset_)

train_indices=[]

data_text=[] #querys train

 #dictionary ground truth retrieval img o text
all_r_img, all_r_text= {},{} #train+val+test
val_r_img, val_r_text, test_r_img, test_r_text= {},{},{},{} 

for i in range(len(dataset)):
  if dataset[i]["split"]=="train":
    train_indices.append(dataset[i]["imgid"])
    for t in dataset[i]["sentids"]:
      data_text.append(t)
      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]


  if dataset[i]["split"]=="val":

    for t in dataset[i]["sentids"]:
      val_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      val_r_img[t]= dataset[i]["imgid"]

      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]

  if dataset[i]["split"]=="test":

    for t in dataset[i]["sentids"]:
      test_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      test_r_img[t]= dataset[i]["imgid"]

      all_r_text[dataset[i]["imgid"]]= dataset[i]["sentids"]
      all_r_img[t]= dataset[i]["imgid"]


print("Train-Val-Test")
print("img: ", len(train_indices),len(val_r_text.keys()), len(test_r_text.keys()))
print("text: ",len(data_text), len(val_r_img.keys()), len(test_r_img.keys()))
print("-"*20)
# captions order for modeling e.g: [id 999] ->[position 0] with .index
universe_text= data_text+ list(val_r_img.keys())+list(test_r_img.keys()) 
#img order for modeling
universe_img= train_indices+list(val_r_text.keys())+ list(test_r_text.keys())
print("All Img-Text")
print(len(universe_img),len(universe_text))



IndentationError: expected an indented block after 'if' statement on line 28 (592475163.py, line 30)

In [50]:
import sklearn.preprocessing

textual_matrix_train= np.load("ROCO/ALL_R_CLIP_text.npy")
print(textual_matrix_train.shape)
textual_matrix_train_last= np.load("ROCO/last_R_CLIP_text_.npy")[-50:,:]
print(textual_matrix_train_last.shape)
textual_matrix_train= np.concatenate((textual_matrix_train,textual_matrix_train_last), axis=0)
textual_matrix_train = np.delete(textual_matrix_train, train_errors, axis=0)
print(textual_matrix_train.shape)

textual_matrix_val= np.load("ROCO/validation_ALL_R_CLIP_text.npy")
print(textual_matrix_val.shape)
textual_matrix_val_last= np.load("ROCO/last_validation_R_CLIP_text_.npy")
print(textual_matrix_val_last.shape)
textual_matrix_val= np.concatenate((textual_matrix_val,textual_matrix_val_last), axis=0)
textual_matrix_val = np.delete(textual_matrix_val, val_errors, axis=0)
print(textual_matrix_val.shape)

textual_matrix_test= np.load("ROCO/TEST_ALL_R_CLIP_text.npy")
print(textual_matrix_test.shape)
textual_matrix_test_last= np.load("ROCO/last_T_R_CLIP_text_.npy")
print(textual_matrix_test_last.shape)
textual_matrix_test= np.concatenate((textual_matrix_test,textual_matrix_test_last), axis=0)
textual_matrix_test = np.delete(textual_matrix_test, test_errors, axis=0)
print(textual_matrix_test.shape)


textual_matrix= np.concatenate((textual_matrix_train,textual_matrix_val,textual_matrix_test), axis=0)


v_matrix_train= np.load("ROCO/ALL_R_CLIP_imgs.npy")

print(v_matrix_train.shape)

v_matrix_val= np.load("ROCO/validation_ALL_R_CLIP_imgs.npy")

print(v_matrix_val.shape)

v_matrix_test= np.load("ROCO/TEST_ALL_R_CLIP_imgs.npy")

print(v_matrix_test.shape)


visual_matrix= np.concatenate((v_matrix_train,v_matrix_val,v_matrix_test), axis=0)

#visual_matrix=sklearn.preprocessing.normalize(visual_matrix[universe_img,:], norm='l2', axis=0) #visual_matrix[universe_img,:]/np.max(visual_matrix)
#textual_matrix=textual_matrix[universe_text,:]#/np.max(textual_matrix)
print(textual_matrix.shape, visual_matrix.shape)




(65400, 512)
(50, 512)
(65419, 512)
(8100, 512)
(80, 512)
(8175, 512)
(8100, 512)
(79, 512)
(8176, 512)
(65419, 512)
(8175, 512)
(8176, 512)
(81770, 512) (81770, 512)


In [79]:
import concurrent.futures
from collections import defaultdict
from scipy.sparse import csr_matrix
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing
import gc

def get_batch_data(data, index, size):
    """
    For minibatch training
    """
  
    column_1 = []
    column_2 = []
    for i in range(index, index + size):
        line = data[i]
        column_1.append(int(line[0]))#text
        column_2.append(int(line[1]))#img
    return np.array(column_1), np.array(column_2)


_model = None
_universe_query= None
_retrieval= None
evaluation_Q_= None
_universe_value= None
_list_rank= None
_at_k= None
evaluation_V_=None
def evaluate_model(model, nDCG, split, retrieval): #i2i
    """
    evaluate model
    """
    global _model
    global _universe_query
    global _universe_value
    global _retrieval
    global evaluation_Q_
    global _list_rank
    global _at_k
    global evaluation_V_

    _list_rank= []
    _retrieval=retrieval
    _model = model
    test_users = []
    results=[]
    output= []
    _at_k=10

    #-----------------------------------------------------------
    if _retrieval=="t2i":
      _universe_query= universe_text
      _universe_value= universe_img
      if split=="val":
        evaluation_Q_= val_r_img
        evaluation_V_= list(val_r_text.keys())
      if split=="test": #test
        evaluation_Q_= test_r_img
        evaluation_V_= list(test_r_text.keys())
      #output.append(t2i( _model,split))

    else:  #i2t
      _universe_query= universe_img
      _universe_value= universe_text
      if split=="val":
        evaluation_Q_= val_r_text
        evaluation_V_= list(val_r_img.keys())
      else: #test
        evaluation_Q_= test_r_text
        evaluation_V_= list(test_r_img.keys())
      #output.append(i2t( _model,split))
    

    evaluation_Q_num = len(evaluation_Q_.keys())
    #print("evaluation_Q_num ",evaluation_Q_num, "_universe_query ",len(_universe_query), "evaluation_V_ ", len(evaluation_V_) )

    result_acumulate= np.array([0.] * 3) # 3 metrics
    for q in evaluation_Q_.keys():
        result_acumulate+=eval_one_user(q,split) 

    ret = result_acumulate / evaluation_Q_num #avg 
    ret = list(ret)

    results.append(ret)
    output.append(results)

    if nDCG: 
            ndcg_i2i= t2i( _model, split ,aux_mod=True)
            print(ndcg_i2i)
            ndcg_t2t=  t2t( _model,split) 
            print(ndcg_t2t)
            ndcg_i2t=i2t( _model,split)
            print(ndcg_i2t)
            ndcg_t2i= t2i( _model, split)
            print(ndcg_t2i)
            output.append((ndcg_i2t,ndcg_t2i, ndcg_i2i,ndcg_t2t))

    return output



def eval_one_user(query,split):
    """
    each evaluation
    """

    rating = _model.predict(_universe_query.index(query), _retrieval,split)
    #gc.collect()
    if _retrieval=="t2i":
      ground_truth = [ evaluation_Q_[query] ] #one img
    else: #i2t
      ground_truth =  evaluation_Q_[query] #list
      #print(query)

    rank_=np.argsort(-rating)
    rank= [evaluation_V_[i]  for i in rank_[:_at_k]]

    result = []
    result.append(recall_at_k(rank, ground_truth, 1))
    result.append(recall_at_k(rank, ground_truth, 5))
    result.append(recall_at_k(rank, ground_truth, 10))


    return np.array(result)



def recall_at_k(rank, ground_truth, k):
    """
    Recall
    """
    #return 100 * len(set(rank[:k]) & set(ground_truth)) / len(ground_truth)
    if len(set(rank[:k]) & set(ground_truth))>=1:
        return 100
    else:
        return 0



In [80]:

    def calculate_diversity(_list_rank, len_q):
        """ 
        max_= len(_universe_value)    
        coverage= set()
        label_binarizer = sklearn.preprocessing.LabelBinarizer()
        label_binarizer.fit(range(max_))
        list_=[]
        dict_count={}
        for rank in _list_rank: # each RANKING
            coverage= coverage | set(rank)
            list_.append( np.sum(label_binarizer.transform(rank),axis=0) )
            for i in rank:
                if i not in dict_count:
                    dict_count[i]=1
                else:
                    dict_count[i]+=1
        recs_content=csr_matrix(np.array(list_) )
        #calculate similarity scores for all items in list
        similarity = cosine_similarity(X=recs_content, dense_output=False)
        #get indicies for upper right triangle w/o diagonal
        upper_right = np.triu_indices(similarity.shape[0], k=1)
        diversity_cos = np.mean(similarity[upper_right])

        """
        _at_k=25
        dict_count={}
        print("novedad: ", _at_k, len_q )
        for rank in _list_rank: # each RANKING
            #coverage= coverage | set(rank)
            #list_.append( np.sum(label_binarizer.transform(rank),axis=0) )
            for i in rank:
                if i not in dict_count:
                    dict_count[i]=1
                else:
                    dict_count[i]+=1
        #novelty-diver
        nov=0
        for rank in _list_rank: # each RANKING
            self_information=0
            for i in rank:
                self_information += np.sum( -np.log2(dict_count[i]/len_q) )
            nov+=self_information/_at_k #top
            #

        return (0., 0., nov/len_q) #(diversity_cos, len(coverage)/max_, nov/len(evaluation_Q_))

    # (C) Mathieu Blondel, November 2013
    # License: BSD 3 clause
    #Original https://github.com/mesnico/TERAN/blob/master/evaluate_utils/dcg.py

    import numpy as np
    import os

    class DCG:
        def __init__(self, rank=25, relevance_methods=['rougeL'], split="val"): # config, n_queries, split
            self.rank = rank
            self.relevance_methods = ["RougeL","PubMedBert"]
            RougeL= np.load("ROCO/similarity_matrix/rouge_matrix.npy")
            PubMedBert= np.load("ROCO/similarity_matrix/PubMedBERT_cosine.npy")
            self.npts=8176 
            self.relevances = [RougeL,PubMedBert]
            # una fila se lleva a una matriz de n_queries filas y lo que quede de columnas (en orden)
        def compute_ndcg(self, query_id, sorted_indexes, fold_index=0, retrieval='t2i'):
            """
            sorted_indexe: all sorted indices
            query_id: image retrieval \in [0,5000], sentence retrieval \in [0,1000]
            # npts: images.shape[0] // 5
            fold_index= for 5fold cross-validation, only for MSCOCO
            """
            sorted_indexes = sorted_indexes[:self.rank] #TOP@K
            if retrieval == 't2i':
                query_base = self.npts * fold_index #0
                #caption query pertenece a  [0,5000] valores de relevancia en vector [0,1000] imagenes
                # (5000,1000)
                relevances = [r[query_base + query_id, fold_index * self.npts : (fold_index + 1) * self.npts] for r in self.relevances]
            elif retrieval == 'i2t':
                query_base = self.npts * fold_index #0
                #  valores de relevancia en vector [0,5000] captions, image query pertenece a  [0,1000] 
                relevances = [r[fold_index * self.npts  : (fold_index + 1) * self.npts , query_base + query_id] for r in self.relevances]
            ndcg_scores = [ ndcg_from_ranking(r, sorted_indexes)  for r in relevances]
            out = {k: v for k, v in zip(self.relevance_methods, ndcg_scores)}
            return out


    def ndcg_from_ranking(y_true, ranking):

        # El orden del muestreo es irrelevante en ndcg_from_ranking o dcg_score:
        # [0.5, 0.3, 0.2], [2, 1, 0] == [0.2, 0.3, 0.5], [0, 1, 2]
        # lo que importan es el mach (relevancia,posición del ranking)

        """Normalized discounted cumulative gain (NDCG) at rank k
        Parameters
        ----------
        y_true : array-like, shape = [n_samples]
            Ground truth (true relevance labels).
        ranking : array-like, shape = [k]
            Document indices, i.e.,
                ranking[0] is the index of top-ranked document,
                ranking[1] is the index of second-ranked document,
                ...
        k : int
            Rank.
        Returns
        -------
        NDCG @k : float
        """
        k = len(ranking)
        best_ranking = np.argsort(y_true)[::-1]#indices de mayor a menor valores
        best = dcg_from_ranking(y_true, best_ranking[:k]) #el mejor posible 
        if best == 0:
            return 0
        return dcg_from_ranking(y_true, ranking) / best

    def dcg_from_ranking(y_true, ranking):
        #Por ejemplo en el caso binario:
        #dcg_from_ranking([1, 0, 0], [2, 1, 0])=0.5= 0+0+1/log_2(4)
        #dcg_from_ranking([1, 0, 0], [0, 1, 2])=1.0= 1/log_2(2)+0+0
        #Además, corta solo para el top@K
        """Discounted cumulative gain (DCG) at rank k
        Parameters
        ----------
        y_true : array-like, shape = [n_samples]
            Ground truth (true relevance labels). #comunmente binario
        ranking : array-like, shape = [k]
            Document indices, i.e.,
                ranking[0] is the index of top-ranked document,
                ranking[1] is the index of second-ranked document,
                ...
        k : int
            Rank.
        Returns
        -------
        DCG @k : float
        """
        y_true = np.asarray(y_true)
        ranking = np.asarray(ranking)
        #ordena y toma el subconjunto de los valores de y_true con respecto al ranking asociado
        rel = y_true[ranking] #normalmente esta es binaria
        gains = 2 ** rel - 1 #en el caso continuo, [0,1] y no {0,1}
        discounts = np.log2(np.arange(len(ranking)) + 2)
        return np.sum(gains / discounts)#div pointwise


    #Original https://github.com/mesnico/TERAN/blob/master/evaluation.py


    #n_queries= 5000 #img son 1000 files son 5000000==5000*1000


    def i2t( model, split, n_ndcg=True, d_n=True, top=25):
        """
        TEXT RETRIEVAL

        model: Dado un img query el modelo debe dar el ranking asociado (largo 5000, de 0 a 4999) en orden de aparición!!!
        splt: "val" or "test"
        n_ndcg: novelty ndcg
        d_n: other list metrics diversity and novelty of lists
        """
        output=[]

        if split=="val":
          eval_indices= list(val_r_text.keys()) #img
          all_indices= all_val_indices_img #original order  
          eval_values= list(val_r_img.keys()) #text
          all_values= all_val_indices_text #original order
        else:#test
          eval_indices= list(test_r_text.keys()) #img
          all_indices= all_test_indices_img #original order 
          eval_values= list(test_r_img.keys())#text
          all_values= all_test_indices_text #original order



        ndcg_scorer = DCG(rank=top, relevance_methods=['rougeL', 'spice'], split=split)
        rougel_ndcgs = np.zeros(ndcg_scorer.npts)
        spice_ndcgs = np.zeros(ndcg_scorer.npts)
        div_rankings=[]

        nov_rankings= []


        for index in range(len(eval_indices)): #range(npts): #each QUERY
            rating= model.predict( universe_img.index(eval_indices[index]) , "i2t", split)      
            if d_n:
                rank_=np.argsort(-rating)
                div_rankings.append( rank_[:top] )
            # positions in order wrt rating (only for evaluation)    
            indices_= np.argsort( -np.array( rating) ) 
            # get original position (in order by the model)
            inds= [all_values.index(eval_values[i]) for i in indices_] #order of values = eval_values
            rougel_ndcgs[index], spice_ndcgs[index] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[index]), inds,retrieval='i2t').values()

            if n_ndcg:
                nov_rankings.append(inds)


        mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
        mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
        output.append( (mean_rougel_ndcg, mean_spice_ndcg) )

        if n_ndcg:
            eval_indices_all=[all_indices.index(i) for i in eval_indices]
            rankings={k: v for k, v in zip(eval_indices_all,nov_rankings)}

            myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="rougeL")
            novedad_rougeL=myAlpha.calculate()

            myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="spice")
            novedad_spice=myAlpha.calculate()

            output.append((novedad_rougeL,novedad_spice))
        if d_n:
            cos_div,coverage,nov= calculate_diversity(div_rankings, len(eval_indices))
            output.append((coverage,cos_div,nov))

        return output


    def t2i( model,split,n_ndcg=True, d_n=True, aux_mod=False,top=25):
        #aux_mod for i2i
        output=[]
        if split=="val":
          eval_indices= list(val_r_img.keys()) #text
          all_indices= all_val_indices_text 
          eval_values= len(val_r_text.keys()) #img
          all_values= all_val_indices_img 
        else:
          eval_indices= list(test_r_img.keys())
          all_indices= all_test_indices_text  
          eval_values= list(test_r_text.keys())
          all_values= all_test_indices_img 


        ndcg_scorer = DCG( rank=top, relevance_methods=['rougeL', 'spice'],split=split)
        rougel_ndcgs = np.zeros(ndcg_scorer.npts)
        spice_ndcgs = np.zeros(ndcg_scorer.npts)

        nov_rankings= []
        div_rankings=[]


        for index in range( len(eval_indices) ): 

            if aux_mod==False: #t2i
                rating= model.predict( universe_text.index(eval_indices[index ]), "t2i", split)
                #i2i
            else:#t->i-> retrieval: each t have a diferent rel wrt each grup of images.
                rating= model.predict( universe_img.index(eval_values[index]) , "i2i",split)

            if d_n:
                rank_=np.argsort(-rating)
                div_rankings.append( rank_[:top] )

            indices_= np.argsort( -np.array(rating) ) 
            inds= [all_values.index(eval_values[j]) for j in indices_] 
            rougel_ndcgs[ index ], spice_ndcgs[ index ] = ndcg_scorer.compute_ndcg(all_indices.index(eval_indices[ index ]), inds, retrieval='t2i').values()
            if n_ndcg:
                nov_rankings.append(inds)



        mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
        mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
        output.append((mean_rougel_ndcg, mean_spice_ndcg))

        if n_ndcg:
            eval_indices_all=[all_indices.index(i) for i in eval_indices]
            rankings={k: v for k, v in zip( eval_indices_all, nov_rankings)}
            myAlpha = AlphaNDCG(rankings= rankings, retrieval= "t2i",ndcg_scorer=ndcg_scorer, method="rougeL")
            novedad_rougeL=myAlpha.calculate()
            myAlpha = AlphaNDCG(rankings= rankings, retrieval= "t2i",ndcg_scorer=ndcg_scorer, method="spice")
            novedad_spice=myAlpha.calculate()
            output.append((novedad_rougeL,novedad_spice))

        if d_n:
            cos_div,coverage,nov= calculate_diversity(div_rankings, len(eval_indices))
            output.append((coverage,cos_div,nov))  
        return output


    def t2t( model, split,n_ndcg=True, d_n=True,top=25):

        output=[]

        if split=="val":
          #eval_indices= list(val_r_text.keys()) #img
          all_indices= all_val_indices_img #img 
          all= all_val_indices_text 
          eval_dic= val_r_img
        else:
          #eval_indices= list(test_r_text.keys()) #img
          all_indices= all_test_indices_img #img 
          all= all_test_indices_text
          eval_dic= test_r_img

        eval_values= list(eval_dic.keys()) # texts


        ndcg_scorer = DCG( rank=top, relevance_methods=['rougeL', 'spice'], split=split)
        rougel_ndcgs = np.zeros(ndcg_scorer.npts)
        spice_ndcgs = np.zeros(ndcg_scorer.npts)
        querys=[]

        nov_rankings= []
        div_rankings=[]
        for index in range( len(eval_values)): 
            rating= model.predict( universe_text.index(eval_values[index]) , "t2t",split)
            if d_n:
                rank_=np.argsort(-rating)
                div_rankings.append( rank_[:top] )


            indices_= np.argsort( -np.array(rating) ) 
            inds= [all.index(eval_values[i]) for i in indices_] 
            #t->i->index
            img_query_from_text=all_indices.index(eval_dic[eval_values[index]]) 
            querys.append(img_query_from_text)
            rougel_ndcgs[index], spice_ndcgs[index] = ndcg_scorer.compute_ndcg( img_query_from_text, inds,retrieval='i2t').values()
            if n_ndcg:
                nov_rankings.append(inds)


        mean_rougel_ndcg = np.mean(rougel_ndcgs[rougel_ndcgs != 0])
        mean_spice_ndcg = np.mean(spice_ndcgs[spice_ndcgs != 0])
        output.append((mean_rougel_ndcg, mean_spice_ndcg))

        if n_ndcg:
            rankings={k: v for k, v in zip(querys,nov_rankings)}
            myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="rougeL")
            novedad_rougeL=myAlpha.calculate()
            myAlpha = AlphaNDCG(rankings= rankings, retrieval= "i2t",ndcg_scorer=ndcg_scorer, method="spice")
            novedad_spice=myAlpha.calculate()
            output.append((novedad_rougeL,novedad_spice))

        if d_n:
            cos_div,coverage,nov= calculate_diversity(div_rankings, len(eval_values))
            output.append((coverage,cos_div,nov)) 
        return output
    
    class AlphaNDCG(object):
     def __init__(self, alpha=0.5, top=25,rankings=None,  retrieval= "i2t",ndcg_scorer=None, method= None):
      self.alpha = alpha
      self.fold_index=0
      self.retrieval= retrieval
      self.all_rankings= rankings
      self.ndcg_scorer= ndcg_scorer
      self.mode= ['rougeL', 'spice'].index(method)
      self.top= top
      self.npts= self.ndcg_scorer.npts


     def relevance(self, query_id):


        if self.retrieval == 't2i':
                    query_base = self.npts  * self.fold_index #0
                    relevances = [r[query_base + query_id, self.fold_index * self.npts : (self.fold_index + 1) * self.npts] for r in self.ndcg_scorer.relevances]
        if self.retrieval == 'i2t':
                    query_base = self.npts * self.fold_index #0
                    relevances = [r[self.fold_index * self.npts  : (self.fold_index + 1) * self.npts , query_base + query_id] for r in self.ndcg_scorer.relevances]

        return relevances[self.mode]


     def top_querys(self, query_id,querys_a_ordenar):   

        if self.retrieval == 't2i':
            value_retrieval= "i2t" #value=img pasa a ser query para retrieval de text
        if self.retrieval == 'i2t':
            value_retrieval= "t2i"

        if value_retrieval == 't2i': #retrieval img quer
                    query_base = self.npts  * self.fold_index #0
                    relevances = [r[query_base + query_id, self.fold_index * self.npts : (self.fold_index + 1) * self.npts] for r in self.ndcg_scorer.relevances]
        if value_retrieval == 'i2t':
                    query_base = self.npts * self.fold_index #0
                    relevances = [r[self.fold_index * self.npts  : (self.fold_index + 1) * self.npts , query_base + query_id] for r in self.ndcg_scorer.relevances]


        return sorted(querys_a_ordenar, key=lambda x: list(relevances[self.mode])[x], reverse= True)[:self.top] 



     def ideal_value(self):
       # normalizarion with top most relevant per query 
      ndcg=0
      for query in self.all_rankings: # each ranking
       ranking= np.argsort(-self.relevance(query))[:self.top] #top by relevance
       metric=0
       i=1

       for value in ranking: #each value
            if i==1: #first 
                sum_numerator=self.relevance(query)[value]
                rel_ac= self.relevance(query)[value]
            else:
                sum_numerator+=self.relevance(query)[value]  * ( (1 - self.alpha)**rel_ac )
                rel_ac+= self.relevance(query)[value]

            metric+= sum_numerator/np.log2(i+1)
            i+=1

       ndcg+=metric

      return ndcg/ len(self.all_rankings) #mean



     def calculate(self):
      ndcg=0
      normalization= self.ideal_value()

      for query in self.all_rankings: # each ranking
        ranking= self.all_rankings[query][:self.top]
        metric=0
        i=1
        for value in ranking: #each value
         if i==1:
            sum_numerator=self.relevance(query)[value]
            rel_ac= self.relevance(query)[value]
         else:
            sum_numerator+=self.relevance(query)[value]  * ( (1 - self.alpha)**rel_ac )
            rel_ac+= self.relevance(query)[value]

         metric+= sum_numerator/np.log2(i+1)
         i+=1

        ndcg+=metric/ normalization

      return ndcg/ len(self.all_rankings) #mean

 

In [95]:
path= "ROCO"
class Net(object):
        def __init__(self, visual_matrix, textual_matrix,factor, margen,mode , reg,pretrain=False):
            self.factor= factor
            self.reg= reg
            self.batch_size =  64 #VSE+
            self.epochs = 30 
            self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
            self.estimator="Linear"
            self.DIS_MODEL_FILE= path+"/"+self.estimator+"/"+mode+"/"+str(margen)+"_"+str(factor)+"_"+str(self.batch_size)+"_"+str(self.reg)
            print(self.DIS_MODEL_FILE)
            self.mode= mode
            self.alpha=margen
            self.phase= "train"
            
                
            if self.estimator=="Linear":

                layer_name = 'Visual_Enconder'
    
                v_input = tf.keras.Input(shape=(visual_matrix.shape[1],))

                v_output = tf.keras.layers.Dense(units=self.factor, use_bias=False )(v_input)
                v_output= tf.keras.layers.UnitNormalization()(v_output) #tf.math.l2_normalize(v_output)  #

                self.visual_encoder = tf.keras.Model(inputs=[v_input], outputs=v_output)


                layer_name = 'Textual_Enconder'
                t_input = tf.keras.Input(shape=(textual_matrix.shape[1],))
                t_output = tf.keras.layers.Dense(units=self.factor,use_bias=False )(t_input)
                t_output= tf.keras.layers.UnitNormalization()(t_output) #tf.math.l2_normalize(t_output)  #

                self.text_encoder= tf.keras.Model(inputs=[t_input], outputs=t_output)


            if pretrain: 

                self.visual_encoder= tf.keras.models.load_model(self.DIS_MODEL_FILE+'visual_encoder_tf')
                self.text_encoder= tf.keras.models.load_model(self.DIS_MODEL_FILE+'text_encoder_tf')
                print("pre train")
            
            
            i_input = tf.keras.Input(shape=(visual_matrix.shape[1],), name="i")
            i_n_input = tf.keras.Input(shape=(visual_matrix.shape[1],), name="i_n")
            c_input = tf.keras.Input(shape=(textual_matrix.shape[1],), name="c")
            c_n_input = tf.keras.Input(shape=(textual_matrix.shape[1],), name="c_n")

             
            i_emb = self.visual_encoder(i_input) 
            i_n_emb = self.visual_encoder(i_n_input)
            c_emb = self.text_encoder(c_input)
            c_n_emb = self.text_encoder(c_n_input)
            
            
            
            i_c= tf.keras.layers.Dot(axes=1)([i_emb, c_emb]) #self.similarity([i_emb,c_emb])
            i_c_n= tf.keras.layers.Dot(axes=1)([i_emb, c_n_emb]) #self.similarity([i_emb,c_n_emb])
            i_n_c= tf.keras.layers.Dot(axes=1)([i_n_emb, c_emb]) #self.similarity([i_n_emb,c_emb])
            
            
            
            i_n_i= tf.keras.layers.Dot(axes=1)([i_n_emb, i_emb]) #self.similarity([i_n_emb,i_emb])
            c_n_c=tf.keras.layers.Dot(axes=1)([c_n_emb, c_emb]) #self.similarity([c_n_emb,c_emb])
            i_n_c_n= tf.keras.layers.Dot(axes=1)([i_n_emb, c_n_emb]) #self.similarity([i_n_emb,c_n_emb])
            
            sim= ( i_c,i_c_n,i_n_c, i_n_i, c_n_c,i_n_c_n)
            
            


            self.model =  tf.keras.Model(inputs=[i_input, i_n_input, c_input,c_n_input], outputs=sim)
            

            self.visual_encoder.compile(optimizer=self.optimizer)
            self.text_encoder.compile(optimizer=self.optimizer)
            self.model.compile(optimizer=self.optimizer)

        

        def loss(self,input_img, img_negative, input_text,text_negative,mask_s_p):

          i_input = tf.nn.embedding_lookup(visual_matrix, input_img) #[_,1280]
          i_n_input = tf.nn.embedding_lookup(visual_matrix, img_negative) #[_,1280]
          c_input = tf.nn.embedding_lookup(textual_matrix, input_text) #[_,1280]
          c_n_input = tf.nn.embedding_lookup(textual_matrix, text_negative) #[_,1280]
          (i_c,i_c_n,i_n_c,i_n_i ,c_n_c,i_n_c_n) = self.model([i_input, i_n_input, c_input,c_n_input])
          
          elements=np.array([[0,0],[0,0]])
          errors=np.array([[0,0],[0,0]])
          if self.phase=="test": 
              elements= (i_c, i_n_c, i_c_n, i_n_i , c_n_c, i_n_c_n)   
              e_1=np.array((elements[0]>elements[3]+self.alpha)).sum() #i_c>i_i_n
              e_2=np.array((elements[0]>elements[4]+self.alpha)).sum() #i_c>c_c_n
              e_3=np.array((elements[0]>elements[5]+self.alpha)).sum() #i_c>i_n_c_n
              errors= (e_1,e_2,e_3)
            
          if self.mode=="F_HN": #full hard negative  
              i2t_t2i_loss= tf.reduce_mean(tf.maximum(self.alpha - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(self.alpha - i_c+i_n_c, 0) )
              v_c_loss=tf.reduce_mean(tf.maximum(self.alpha- i_c+i_n_i, 0) )
              t_c_loss= tf.reduce_mean(tf.maximum(self.alpha - i_c+c_n_c, 0) )
              s_p_loss= tf.reduce_mean(tf.multiply(mask_s_p,tf.maximum(self.alpha - i_c+i_n_c_n, 0)))

              losses= i2t_t2i_loss + self.reg*(v_c_loss + t_c_loss+ s_p_loss)
                
          if self.mode=="HN" or self.mode=="random": # hard negative
              i2t_t2i_loss= tf.reduce_mean(tf.maximum(self.alpha - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(self.alpha - i_c+i_n_c, 0) )
              losses= i2t_t2i_loss 
              
            
          if self.mode=="M_HN": # margen hard negative
              i2t_t2i_loss= tf.reduce_mean(tf.maximum(i_n_i - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(c_n_c - i_c+i_n_c, 0) )
              #i2t_t2i_loss= tf.reduce_mean(tf.maximum(c_n_c - i_c+i_c_n, 0) )+ tf.reduce_mean(tf.maximum(i_n_i - i_c+i_n_c, 0) )

              losses= i2t_t2i_loss 
          
          return losses,elements,errors
        

        def predict(self,query, retrieval,split):
            if type(query) is not list:
                query= [query]
            if retrieval=="t2i":
                c_input = self.text_encoder(tf.nn.embedding_lookup(textual_matrix, query))
                if split=="val":
                    i_eval= self.visual_encoder(visual_matrix[ self.val_indices_])
                if split=="test":
                    i_eval= self.visual_encoder(visual_matrix[self.test_indices_])
                if split=="train":
                    i_eval=self.visual_encoder(visual_matrix[self.train_indices_]) 
                
                rating= tf.matmul(i_eval, tf.transpose(c_input))
            
            if retrieval=="i2t": 
                i_input = self.visual_encoder(tf.nn.embedding_lookup(visual_matrix, query))
                if split=="val":
                    c_eval= self.text_encoder(textual_matrix[self.data_text_Val])
                if split=="test":
                    c_eval= self.text_encoder(textual_matrix[self.data_text_Test])
                if split=="train":
                    c_eval=self.text_encoder(textual_matrix[self.data_text_])                
        
                rating= tf.matmul(i_input, tf.transpose(c_eval) )

            if retrieval=="i2i":
                if split=="test":
                    i_query = self.visual_encoder(tf.nn.embedding_lookup(visual_matrix, query))
                    i_eval=  self.visual_encoder(visual_matrix[self.test_indices_]) 
                    rating= tf.matmul(i_query, tf.transpose(i_eval) )
            if retrieval=="t2t":
                if split=="test":
                    c_query = self.text_encoder(tf.nn.embedding_lookup(textual_matrix, query))
                    c_eval= self.text_encoder(textual_matrix[self.data_text_Test])
                    rating= tf.matmul(c_eval , tf.transpose(c_query)  )

            return np.reshape(rating, [-1])        
                

        def save(self, model_time, loss_c,recalls,epoch, best_recall ):
          with open(self.DIS_MODEL_FILE+'train.txt', 'w') as f:
            f.write("training time of model: %fs" % (model_time) + "\n")
            f.write("finish epoch %s" % epoch + "with recall %s" % best_recall  + "\n")
          np.save(self.DIS_MODEL_FILE+"Loss_recalls.npy", np.array([loss_c,recalls],dtype=object))
          
          self.visual_encoder.save(self.DIS_MODEL_FILE+'visual_encoder_tf',save_format='tf')
          self.text_encoder.save(self.DIS_MODEL_FILE+'text_encoder_tf',save_format='tf')


        def training(self):
           
            
            data = []
            self.data_text_=[]

            self.val_indices_=  [ universe_img.index(i) for i in val_r_text.keys() ]
            self.train_indices_= [universe_img.index(i) for i in train_indices]
            self.data_text_Val=  [ universe_text.index(i) for i in val_r_img.keys() ]

            for i in self.train_indices_:
                for t in all_r_text[ train_indices[self.train_indices_.index(i)] ]:
                    data.append([ universe_text.index(t), i ]) # [texto, img]
                    self.data_text_.append(universe_text.index(t))
    

            recalls=[]
            loss_c=[]
            best_recall= 0
            start_time = time.time() 

            train_size = len(data)
            c_images= len(self.train_indices_)
            
                

            for epoch in range(self.epochs):
                
        
                if epoch>=15:
                    self.optimizer.learning_rate.assign(0.00002)
                    
                print("epoch: %d" % epoch)
                np.random.shuffle(data)
                index = 0
                loss_c_=[]
                loss_elements=[]
                loss_errores=[]
                

                while index + self.batch_size < train_size:
                    #print((index + self.batch_size)/train_size)
                    input_text, input_img = get_batch_data(data, index,self.batch_size)  
                    index += self.batch_size
                    mask_s_p= [0]*self.batch_size
                    
                    
                    if self.mode=="random": #train quitando - minibatch (positivos)
                      text_negative= np.random.choice( list( set( self.data_text_ )- set(input_text) ) , self.batch_size, replace=False)
                      img_negative= np.random.choice(list( set(self.train_indices_) - set(input_img) ), self.batch_size, replace=False)
                    
                    else: 
                        img_negative,text_negative=[],[] #negatives
                        
                        rating= np.array( self.predict( list(input_text) ,"t2i", "train"), dtype=float)
                        gc.collect() 
                        rating_text= np.array(self.predict( list(input_img) ,"i2t", "train"), dtype=float)
                        gc.collect()
                        
                        for i_ in range(self.batch_size):
                            ratings_batch= rating[c_images*i_:c_images*(i_+1)] # t2i query minibatch
                            ratings_text_batch= rating_text[c_images*i_:c_images*(i_+1)]  # i2t query minibatch
                    
                            ratings_batch[train_indices.index(universe_img[input_img[i_]])]= - np.infty
                            five_text= all_r_text[all_r_img[ universe_text[input_text[i_]]] ]
                            ratings_text_batch[ [ data_text.index(j) for j in five_text ] ]= - np.infty

                            img_negative.append(self.train_indices_[np.argmax(ratings_batch)])
                            text_negative.append(self.data_text_[np.argmax(ratings_text_batch)])
                                             #sp condition
                            if  self.mode=="F_HN" and all_r_img[ universe_text[text_negative[-1] ]]  == universe_img[img_negative[-1]] :
                                mask_s_p[i_]=1
                            del ratings_batch
                            del ratings_text_batch
                    
                    with tf.GradientTape() as tape:
                            losses, elements, errors= self.loss(input_img, img_negative, input_text,text_negative, mask_s_p)
                            loss_elements.append( [np.mean(i,axis=0) for i in elements ] )
                            loss_errores.append(errors)
                            
                    grads = tape.gradient(losses, self.model.trainable_variables)
                    self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
                    
                    
                #print(time.time() - start_time)

                
                loss_c.append(( losses.numpy(), np.mean(loss_elements,axis=0),  np.sum(loss_errores,axis=0)))

                if  epoch%5 ==0 or epoch==29:
                    print("validation")
                    r_t2i= evaluate_model(self,nDCG=False, split="val", retrieval="t2i")[0][0]
                    r_i2t= evaluate_model(self,nDCG=False, split="val", retrieval="i2t")[0][0]

                    r_sum=np.sum(r_t2i)  +  np.sum(r_i2t) 


                    print( "Recall t2i-i2t: ", list(r_t2i)+list(r_i2t))
                    recalls.append((r_t2i,r_i2t, r_sum))

                    if r_sum-np.sum(best_recall)>0.1:
                        best_recall=list(r_t2i)+list(r_i2t)
                        wait_for = 0
                        self.save( start_time , loss_c,recalls,epoch,best_recall)

                    else:
                        wait_for += 1
                        if wait_for == 3 +1:
                          print("break train")
                          break
        def inference(self):
            self.test_indices_= [universe_img.index(i) for i in test_r_text.keys()]
            self.data_text_Test= [universe_text.index(i) for i in test_r_img.keys()]

            print("inference")
            results= evaluate_model(self, nDCG=False, split="test", retrieval="i2t")
            print(results)
            results_2= evaluate_model(self, nDCG=True, split="test", retrieval="t2i")
            np.save(self.DIS_MODEL_FILE+"test_metrics.npy", np.array([results,results_2], dtype=object) )
            print(results_2)

F_HN con *10 de lr  0.0002, 0.002 batch size 512

/0.3_1024_256 con /10 lr 0.00002, 0.0002 

In [96]:
net =  Net(visual_matrix,textual_matrix,1024,0.3,"F_HN", 1) 
net.training()

ROCO/Linear/F_HN/0.3_1024_64_1
epoch: 0
validation


Recall t2i-i2t:  [3.840978593272171, 12.892966360856269, 20.34250764525994, 3.0091743119266057, 10.324159021406729, 16.73394495412844]
INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1text_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1text_encoder_tf\assets


...

Recall t2i-i2t:  [5.834862385321101, 17.10091743119266, 25.1131498470948, 3.6452599388379205, 12.207951070336392, 19.25382262996942]
INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1text_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_64_1text_encoder_tf\assets


epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
Recall t2i-i2t:  [5.859327217125382, 16.978593272171253, 25.027522935779817, 3.694189602446483, 12.318042813455657, 19.2782874617737]


In [94]:
for reg in [0.0001,0.001,0.01,0.1]:
    net =  Net(visual_matrix,textual_matrix,1024,0.3,"F_HN", reg) 
    net.training()

ROCO/Linear/F_HN/0.3_1024_512_0.0001
epoch: 0
validation


Recall t2i-i2t:  [2.1529051987767582, 7.131498470948012, 12.452599388379205, 2.6299694189602447, 9.284403669724771, 15.327217125382264]
INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.0001visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.0001visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.0001text_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.0001text_encoder_tf\assets


...

Recall t2i-i2t:  [3.7308868501529053, 12.0, 18.458715596330276, 4.146788990825688, 12.34250764525994, 19.058103975535168]
INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.1visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.1visual_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.1text_encoder_tf\assets


INFO:tensorflow:Assets written to: ROCO/Linear/F_HN/0.3_1024_512_0.1text_encoder_tf\assets


epoch: 26
epoch: 27
epoch: 28
epoch: 29
validation
Recall t2i-i2t:  [3.6819571865443423, 11.926605504587156, 18.605504587155963, 4.0244648318042815, 12.477064220183486, 19.033639143730888]


In [82]:
class zshot(object):
    def __init__(self):
            print("ok")
            
    def predict(self,query, retrieval,split):
            if type(query) is not list:
                query= [query]
            if retrieval=="t2i":
                c_input = tf.nn.embedding_lookup(textual_matrix, query)
                if split=="test":
                    i_eval= visual_matrix[self.test_indices_]
                
                rating= tf.matmul(i_eval, tf.transpose(c_input))
            
            if retrieval=="i2t": 
                i_input = tf.nn.embedding_lookup(visual_matrix, query)
                if split=="test":
                    c_eval= textual_matrix[self.data_text_Test]            
        
                rating= tf.matmul(i_input, tf.transpose(c_eval) )

            if retrieval=="i2i":
                if split=="test":
                    i_query = tf.nn.embedding_lookup(visual_matrix, query)
                    i_eval=  visual_matrix[self.test_indices_]
                    rating= tf.matmul(i_query, tf.transpose(i_eval) )
            if retrieval=="t2t":
                if split=="test":
                    c_query = tf.nn.embedding_lookup(textual_matrix, query)
                    c_eval= textual_matrix[self.data_text_Test]
                    rating= tf.matmul(c_eval , tf.transpose(c_query)  )

            return np.reshape(rating, [-1]) 
        
    def inference(self):
        self.test_indices_= [universe_img.index(i) for i in test_r_text.keys()]
        self.data_text_Test= [universe_text.index(i) for i in test_r_img.keys()]

        print("inference")
        #results= evaluate_model(self, nDCG=False, split="test", retrieval="i2t")
        #print(results)
        results= evaluate_model(self, nDCG=True, split="test", retrieval="t2i")
        np.save("ZS_ROCO_test_metrics.npy", np.array([results], dtype=object) )
        print(results)

In [89]:
#net =  zshot() 
#net.inference()

for name_model in ["random", "HN","M_HN"]:
    net =  Net(visual_matrix,textual_matrix,1024,0.3,name_model) 
    net.inference()

ROCO/Linear/random/0.3_1024_512
inference
[[[0.03669275929549902, 0.09784735812133072, 0.2079256360078278]]]
novedad:  25 8176
[(0.5922726576209263, 0.6656448239672312), (0.8519166220671717, 0.9303335519638007), (0.0, 0.0, 7.9968252935706055)]
novedad:  25 8176
[(0.7188564752022013, 0.8259122077651647), (0.9200992759338698, 0.9711945174598398), (0.0, 0.0, 7.780870660164208)]


KeyboardInterrupt: 